# Cleansing & Exploratory Data Analysis of the Historic Customer data

### Load the historic data

In [ ]:
import pandas as pd

In [5]:
df = pd.read_parquet('../data/data.parquet.gzip', engine='pyarrow')

In [7]:
df

,timestamp,country_code,last_order_ts,first_order_ts,total_orders,voucher_amount
0,2020-05-20 15:43:38.364972+00:00,China,2020-04-19 00:00:00+00:00,2020-04-18 00:00:00+00:00,0.0,5720.0
1,2020-05-20 15:43:47.914346+00:00,China,2020-04-19 00:00:00+00:00,2020-04-13 00:00:00+00:00,1.0,8800.0
2,2020-05-20 15:36:56.866239+00:00,China,2020-04-19 00:00:00+00:00,2020-04-15 00:00:00+00:00,0.0,1760.0
3,2020-05-20 15:00:33.694108+00:00,China,2020-04-19 00:00:00+00:00,2020-04-13 00:00:00+00:00,,1760.0
4,2020-05-20 15:43:18.521402+00:00,China,2020-04-19 00:00:00+00:00,2020-04-14 00:00:00+00:00,3.0,8800.0
...,...,...,...,...,...,...
511422,2020-04-03 18:31:36.635072+00:00,Latvia,2020-01-04 00:00:00+00:00,2019-10-30 00:00:00+00:00,27.0,4400.0
511423,2020-04-03 18:33:32.365196+00:00,Latvia,2020-01-04 00:00:00+00:00,2019-08-29 00:00:00+00:00,10.0,4400.0
511424,2020-04-03 19:23:07.711424+00:00,Peru,2020-01-04 00:00:00+00:00,2019-10-14 00:00:00+00:00,47.0,4400.0
511425,2020-04-03 19:19:06.162203+00:00,Australia,2020-01-04 00:00:00+00:00,2019-09-24 00:00:00+00:00,2.0,11000.0


In [8]:
df.columns

Index(['timestamp', 'country_code', 'last_order_ts', 'first_order_ts',
       'total_orders', 'voucher_amount'],
      dtype='object')

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511427 entries, 0 to 511426
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   timestamp       511427 non-null  datetime64[ns, UTC]
 1   country_code    511427 non-null  object             
 2   last_order_ts   511427 non-null  datetime64[ns, UTC]
 3   first_order_ts  511427 non-null  datetime64[ns, UTC]
 4   total_orders    511427 non-null  object             
 5   voucher_amount  511427 non-null  float64            
dtypes: datetime64[ns, UTC](3), float64(1), object(2)
memory usage: 23.4+ MB


## Some rules and Assumptions
1. The columns containing Total number of orders and Voucher amount are to assumed to be Integer if not set so. 
2. Every column containing timestamps is converted to Datetime format if they were strings before.
3. It is assumed that the Voucher amount is already transformed to a common currency.


### Analyse the Missing Data

In [31]:
df.isna().any()

timestamp         False
country_code      False
last_order_ts     False
first_order_ts    False
total_orders      False
voucher_amount     True
dtype: bool

In [32]:
df.isnull().sum()

timestamp             0
country_code          0
last_order_ts         0
first_order_ts        0
total_orders          0
voucher_amount    29567
dtype: int64

#### 
1. We learn that the voucher_amount column has missing values. Since this corresponds to an Amount, it can be considered a voucher of 0 value and imputed accordingly. 
2. No other Column has any missing values. 

In [37]:
df['voucher_amount'] = df['voucher_amount'].fillna(0)

In [39]:
df.isna().any()

timestamp         False
country_code      False
last_order_ts     False
first_order_ts    False
total_orders      False
voucher_amount    False
dtype: bool

### Convert Columns carrying numbers to Float datatype

In [61]:
df['total_orders'].count()

511427

In [70]:
df['total_orders'][0:10]

0    0.0
1    1.0
2    0.0
3       
4    3.0
5    2.0
6    1.0
7    0.0
8    4.0
9    5.0
Name: total_orders, dtype: object

In [79]:
df['total_orders'][3]
#4th item contains empty strings. Replace it to integer 0

''

In [87]:
def to_float(col):
    if col == '' or col is None:
        return float(0)
    return float(col)

In [88]:
df['total_orders'] = df.apply(lambda x: to_float(x['total_orders']),axis=1)

In [89]:
df['voucher_amount'] = df.apply(lambda x: to_float(x['voucher_amount']),axis=1)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511427 entries, 0 to 511426
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   timestamp       511427 non-null  datetime64[ns, UTC]
 1   country_code    511427 non-null  object             
 2   last_order_ts   511427 non-null  datetime64[ns, UTC]
 3   first_order_ts  511427 non-null  datetime64[ns, UTC]
 4   total_orders    511427 non-null  float64            
 5   voucher_amount  511427 non-null  float64            
dtypes: datetime64[ns, UTC](3), float64(2), object(1)
memory usage: 23.4+ MB


### Convert Strings TS to Datetime Format

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511427 entries, 0 to 511426
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   timestamp       511427 non-null  object             
 1   country_code    511427 non-null  object             
 2   last_order_ts   511427 non-null  object             
 3   first_order_ts  511427 non-null  datetime64[ns, UTC]
 4   total_orders    511427 non-null  object             
 5   voucher_amount  511427 non-null  float64            
dtypes: datetime64[ns, UTC](1), float64(1), object(4)
memory usage: 23.4+ MB


In [54]:
for col in ["timestamp", "last_order_ts", "first_order_ts"]:
    df[col] = pd.to_datetime(df[col])

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 511427 entries, 0 to 511426
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   timestamp       511427 non-null  datetime64[ns, UTC]
 1   country_code    511427 non-null  object             
 2   last_order_ts   511427 non-null  datetime64[ns, UTC]
 3   first_order_ts  511427 non-null  datetime64[ns, UTC]
 4   total_orders    511427 non-null  object             
 5   voucher_amount  511427 non-null  float64            
dtypes: datetime64[ns, UTC](3), float64(1), object(2)
memory usage: 23.4+ MB


### Analyse the country code column

In [94]:
df['country_code'].drop_duplicates()
# We have data from 4 different countries

0          China
5           Peru
72     Australia
123       Latvia
Name: country_code, dtype: object

In [104]:
df['country_code'].value_counts()
#Almost equal geographical distribution

China        169722
Australia    134004
Peru         106547
Latvia       101154
Name: country_code, dtype: int64

## TODO: MAX, MIN Vouchers


### Dump cleansed data

In [96]:
df.to_csv('../data/clean_data.csv')